In [1]:
import pickle
import re
from tqdm import tqdm
import binascii
import numpy as np
import random
import pandas as pd

In [121]:
news = pd.DataFrame()

In [122]:
news_temp = pd.read_csv("data/24.csv", index_col=0)

In [124]:
news_temp = pd.read_csv("data/lenta.csv", index_col=0)

In [126]:
news_temp = pd.read_csv("data/vesti.csv", index_col=0)

In [127]:
news = pd.concat([news, news_temp])

In [128]:
news.shape

(8368, 4)

In [132]:
news.iloc[5287]

date                                            2019.05.19
url            https://www.vestifinance.ru/articles/119411
title    Россия может принять решение о выдаче российск...
text     Прямой ЭфирЗапрет Китая на импорт отходов ожив...
Name: 22, dtype: object

In [100]:
news.tail()

,date,url,title,text
3014,2019.04.21,https://vesti.ru/doc.html?id=3148849&cid=680,"Тарасенко помог ""Сент-Луису"" разгромить ""Сан-Х...",StLouisBlues/Twitter В Сан-Хосе завершилась п...
3015,2019.04.21,https://vesti.ru/doc.html?id=3148847&cid=8,В Уганде перевернулась лодка: два человека пог...,"Global Look Press В воскресенье, 19 мая, на о..."
3016,2019.04.21,https://vesti.ru/doc.html?id=3148846&cid=680,Илья Воробьев: немного переудалялись в игре со...,Тема: twitter.com Главный тренер сборной Росси...
3017,2019.04.21,https://vesti.ru/doc.html?id=3148838&cid=5,"Штаб Зеленского обещает ""фишечки"" на церемонии...",Тема:В Киеве все выходные готовились к инаугур...
3018,2019.04.21,https://vesti.ru/doc.html?id=3148843&cid=8,Взрыв гранаты во время концерта в Колумбии: се...,Global Look Press На северо-востоке Колумбии ...


In [131]:
news.dropna(subset=["text"], inplace=True)

In [113]:
news = news.to_dict("records")

In [2]:
with open("data/all_news.pkl", 'rb') as f:
    news = pickle.load(f)

In [3]:
def get_shingles(words, n=2):
    for i in range (0,len(words)-n+1):
        yield ' '.join(words[i:i+n])

In [4]:
def clean(source):
    stop_symbols = '.,!?:;-\n\r()—'

    stop_words = ('это', 'как', 'так',
                  'и', 'в', 'над',
                  'к', 'до', 'не',
                  'на', 'но', 'за',
                  'то', 'с', 'ли',
                  'а', 'во', 'от',
                  'со', 'для', 'о',
                  'же', 'ну', 'вы',
                  'бы', 'что', 'кто',
                  'он', 'она')

    return [x for x in [y.strip(stop_symbols) for y in source.lower().split()] if x]

In [5]:
shingle_sets = {}
all_shingles = set()
i = 0
for document in tqdm(news):
    shingles_in_doc = set()
    clean_doc = clean(document['body'])
    shingles = {i for i in get_shingles(clean_doc, 5)}
    for shingle in shingles:
        crc = binascii.crc32(shingle.encode('utf-8')) & 0xffffffff
        shingles_in_doc.add(crc)
        all_shingles.add(crc)
    
    shingle_sets[i] = shingles_in_doc
    i += 1

100%|██████████| 20016/20016 [00:05<00:00, 3919.71it/s]


In [6]:
all_shingles = list(all_shingles)

In [7]:
docs_num = len(news)

In [8]:
sim_matrix = np.zeros((docs_num, docs_num))

In [9]:
for i in tqdm(range(docs_num)):
    doc_1 = shingle_sets[i]
    for j in range(i+1, docs_num):
        doc_2 = shingle_sets[j]
        try:
            sim = len(doc_1.intersection(doc_2)) / len(doc_1.union(doc_2))
            sim_matrix[i][j] = sim
            sim_matrix[j][i] = sim
        except:
            sim_matrix[i][j] = 0
            sim_matrix[j][i] = 0

100%|██████████| 20016/20016 [48:40<00:00,  6.85it/s]  


In [10]:
threshold = 0.6

In [11]:
sims = []
duplicates = []
is_break = False
for i in range(docs_num):
    for j in range(i+1, docs_num):
        sim = sim_matrix[i][j]

        if sim > threshold:
            duplicates.append((i, j, sim))

In [12]:
duplicates

[(12702, 14317, 0.624113475177305)]

In [13]:
news[12702]

{'author': None,
 'body': 'Посол России в Минске Дмитрий Мезенцев озвучил стратегию своей работы. Его слова приводит портал TUT.by. «У каждого посла единая задача — сделать все, чтобы отношения Белоруссии и России становились результативнее, предметнее и успешнее, чтобы споров было поменьше, а взаимной, успешной, результативной, обращенной в будущее работы было побольше», — заявил посол. Кроме того, Мезенцев сообщил, что считает Белоруссию «сплоченной, гордой и уверенной страной, огромным другом России, союзником и партнером». Он призвал делать все для того, чтобы укреплять узы этой дружбы. Мезенцев был назначен на пост чрезвычайного и уполномоченного посла России в Белоруссии 30 апреля, сразу после отставки Михаила Бабича, который также был специальным представителем президента России. СМИ писали, что решение об отставке Бабича было принято после неоднократных просьб, которые адресовал лично российскому лидеру Владимиру Путину президент Белоруссии Александр Лукашенко. С момента назнач

In [14]:
news[14317]

{'author': None,
 'body': 'Новый чрезвычайный и полномочный посол России в Белоруссии Дмитрий Мезенцев прибыл на свое место работы в Минск. Об этом сообщает ТАСС со ссылкой на пресс-службу российского посольства в Белоруссии. Мезенцев был назначен на пост чрезвычайного и уполномоченного посла России в Белоруссии 30 апреля, сразу после отставки Михаила Бабича, который также был специальным представителем президента России. СМИ писали, что решение об отставке Бабича было принято после неоднократных просьб, которые адресовал лично российскому лидеру Владимиру Путину президент Белоруссии Александр Лукашенко. С момента назначения Бабича на пост посла в августе 2018 года, белорусская сторона неоднократно выражала недовольство его деятельностью. В частности, Лукашенко обвинял дипломата в том, что он дезинформировал журналистов по вопросу стоимости совместного с Москвой проекта АЭС, а МИД Белоруссии заявлял, что Бабич разрушает отношения между двумя странами. Тем не менее в Кремле сообщали, чт

## MinHashing

In [14]:
num_hashes = 25
sim_matrix = np.zeros((docs_num, docs_num))

In [15]:
def build_new_hashes():
    new_list = []
    for i in range(num_hashes):
        r = random.sample(all_shingles, len(all_shingles))
        new_list.append(r)
        
    return new_list

In [16]:
def calculate_hash(matrix, shingles):
    for i in range(len(matrix)):
        if matrix[i] in shingles:
            return i

In [17]:
hashes = build_new_hashes()

In [18]:
signatures = []

In [19]:
for i in tqdm(range(docs_num)):
    signature = []
    shingles_set = shingle_sets[i]
    
    for m in hashes:
        cur_hash = calculate_hash(m, shingles_set)
        signature.append(cur_hash)
        
    signatures.append(signature)

100%|██████████| 20016/20016 [31:03<00:00, 10.74it/s]  


In [20]:
sim_matrix = np.zeros((docs_num, docs_num))

In [21]:
for i in tqdm(range(docs_num)):
    signature1 = signatures[i]
    for j in range(i + 1, docs_num):
        signature2 = signatures[j]

        count = 0
        for k in range(num_hashes):
            count += signature1[k] == signature2[k]

        sim_matrix[i][j] = (count / num_hashes)
        sim_matrix[j][i] = (count / num_hashes)

100%|██████████| 20016/20016 [16:19<00:00, 20.43it/s] 


In [24]:
threshold = 0.6

In [26]:
sims = []
duplicates = []
is_break = False
for i in range(docs_num):
    for j in range(i+1, docs_num):
        sim = sim_matrix[i][j]

        texts = []
        if sim > threshold:
            duplicates.append((i, j, sim))
#             texts.append(news[j]['text'])

#         if len(texts) > 0:
#             sims.append((news[i]['text'], texts))

In [27]:
duplicates

[(10050, 10288, 0.72),
 (10081, 10144, 0.68),
 (10930, 10933, 0.64),
 (10930, 10938, 0.76),
 (12589, 12594, 0.76),
 (12817, 14622, 0.64),
 (17385, 17489, 0.68)]

In [30]:
news[10081]

{'author': None,
 'body': 'Иран не будет вести переговоры в случае сохранения санкций против него. Такое заявление сделал в Twitter советник президента Ирана Хесамоддин Ашна. «Заявление Америки о готовности вести переговоры с сохранением угроз и санкций неприемлемо», — написал Ашна. Он также отметил, что Тегеран не хочет войны, но и не может считать приемлемым санкционный режим. 23 июня президент США Дональд Трамп заявил, что возможная новая ядерная сделка с Ираном может быть заключена в разных формах, но она в любом случае должна предотвратить появление у Исламской Республики ядерного оружия. 21 июня Трамп сообщил, что остановил удар Вооруженных сил США по Ирану за десять минут до того, как он должен был произойти. Через день он опроверг собственные слова и пояснил, что лишь на время остановил атаку. 20 июня ВВС Корпуса стражей исламской революции сбили беспилотный разведывательный аппарат США. Американская сторона сначала опровергла это сообщение, но затем признала, что Иран сбил БПЛ

In [31]:
news[10144]

{'author': None,
 'body': 'США будут готовы отменить санкции против Ирана в случае заключения новой всеобъемлющей сделки. Такое заявление сделал спецпредставитель США по Ирану Брайан Хук, передает Reuters. «Новая сделка должна быть действительно всеобъемлющей, включая решения ядерной проблемы, проблемы баллистических ракет, прав человека», — отметил Хук. Он добавил, что только в таком случае США смогут полностью отменить санкции и восстановить дипломатические отношения с Ираном. В данный же момент, подчеркнул Хук, Вашингтон продолжит экономическое давление на Тегеран. При этом США оставляют возможность провести переговоры. «Иран знает, как с нами связаться», — сказал спецпредставитель. Ранее советник президента Ирана Хесамоддин Ашна заявил, что Иран не будет вести переговоры в случае сохранения санкций против него. 23 июня президент США Дональд Трамп заявил, что возможная новая ядерная сделка с Ираном может быть заключена в разных формах, но она в любом случае должна предотвратить появ